In [ ]:
#so we're going to be loading a wavpak file

In [7]:
import sys
sys.path.insert(0, "/home/chowder/Documents/unifiedSensorClient")

import platformUtils.logUtils  # register TRACE level and Logger.trace
from writers.wavpakOutput import wavpak_output
import pandas as pd
from datetime import datetime, timezone
import logging

# Print logs from wavpak_output in this notebook
logging.basicConfig(
    level=logging.DEBUG,  # use "trace" level by setting debug_lvl="trace" below
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True,
)

file_name = "20260113T072328p000Z_20260113T133234p000Z.wv"
start_dt = datetime.strptime(file_name.split("_")[0], "%Y%m%dT%H%M%Sp%fZ").replace(tzinfo=timezone.utc)

descriptor = "c57d828b-e8d1-433b-ad79-5420d2136d3f_serial-0_cdtoptech-PA1616S_3d-fix_wgs84_int32-f23_wavpak-3_variable"
file_base_path = "/home/chowder/Documents/unifiedSensorClient/validation/exampleGPSData/" \
    + descriptor + "/" + start_dt.strftime("%Y/%m/%d/")

file_path = file_base_path + file_name

topic = "serial-0_cdtoptech-PA1616S_3d-fix_wgs84_int32-f23_wavpak-3_variable"
output_hz = "variable"
debug_lvl = 20

#instantiate a wavpak output and call load_file
additional_output_config = {
    "input_dtype_str": "float32",
    "wv_dtype_str": "int32",
    "float_bits": 23,
    "bits": 32,
    "sign": "s",
    "channels": 3
}

wavpak_output_obj = wavpak_output( 
            output_base=topic,
            output_hz=output_hz, #variable hz
            temp_write_location="/tmp/",
            debug_lvl=debug_lvl,
            **additional_output_config)
    
timestamps, data_array = wavpak_output_obj.load_file(file_path)

2026-01-13 15:06:42,248 INFO serial-0_cdtoptech-PA1616S_3d-fix_wgs84_int32-f23_wavpak-3_variable_wavpak-output:  starting
2026-01-13 15:06:42,249 INFO serial-0_cdtoptech-PA1616S_3d-fix_wgs84_int32-f23_wavpak-3_variable_wavpak-output: s 32


In [8]:
# generate a dataframe based on timestamps (int64 ns) and lat/lon/alt data
import numpy as np

colNames = ["sampleDT", "lat", "lon", "wgsAltKm"]

# ensure data_array is shaped (n_samples, 3)
if data_array.ndim == 1:
    if data_array.size % 3 != 0:
        raise ValueError("data_array length is not a multiple of 3; cannot form lat/lon/wgsAltKm triples")
    data_reshaped = data_array.reshape(-1, 3)
elif data_array.ndim == 2:
    if data_array.shape[1] >= 3:
        data_reshaped = data_array[:, :3]
    elif data_array.shape[1] == 1 and (data_array.size % 3 == 0):
        data_reshaped = data_array.reshape(-1, 3)
    else:
        raise ValueError("data_array has incompatible shape for lat/lon/wgsAltKm: " + str(data_array.shape))
else:
    raise ValueError("Unexpected data_array ndim: " + str(data_array.ndim))

# align lengths in case timestamps and data differ slightly
n = min(len(timestamps), len(data_reshaped))
if n == 0:
    raise ValueError("No samples to build DataFrame")

sampleDT = pd.to_datetime(timestamps[:n], unit='ns', utc=True)
df = pd.DataFrame(data_reshaped[:n], columns=colNames[1:])
df.insert(0, "sampleDT", sampleDT)

print(f"DataFrame shape: {df.shape}")
#print(df.head())


DataFrame shape: (15881, 4)


In [10]:
df

,sampleDT,lat,lon,wgsAltKm
0,2026-01-13 07:23:28+00:00,256.0,-256.0,256.0
1,2026-01-13 07:23:29+00:00,256.0,-256.0,256.0
2,2026-01-13 07:23:30+00:00,256.0,-256.0,256.0
3,2026-01-13 07:23:31+00:00,256.0,-256.0,256.0
4,2026-01-13 07:23:32+00:00,256.0,-256.0,256.0
...,...,...,...,...
15876,2026-01-13 13:32:30+00:00,256.0,-256.0,256.0
15877,2026-01-13 13:32:31+00:00,256.0,-256.0,256.0
15878,2026-01-13 13:32:32+00:00,256.0,-256.0,256.0
15879,2026-01-13 13:32:33+00:00,256.0,-256.0,256.0


In [9]:
#now with the dataframe with date`time, lat, long, and alt, we can start to generate the gpx file
output_base = "/home/chowder/Documents/unifiedSensorClient/validation/exampleGPSData/exampleGPSData"
output_file_name = "output.gpx"

#we'll be using gpxpy to generate the gpx file
import gpxpy
import gpxpy.gpx

# Build GPX
gpx = gpxpy.gpx.GPX()
track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(track)
segment = gpxpy.gpx.GPXTrackSegment()
track.segments.append(segment)

# Prepare data: drop rows with missing lat/lon; convert altitude to meters
df_gpx = df.dropna(subset=["lat", "lon"]).copy()
alt_m = None
if "wgsAltKm" in df_gpx.columns:
    alt_m = (df_gpx["wgsAltKm"].astype(float) * 1000.0).to_numpy()

# Add points efficiently
for i, row in enumerate(df_gpx.itertuples(index=False)):
    # row order: sampleDT, lat, lon, wgsAltKm
    ts = row.sampleDT.to_pydatetime() if hasattr(row.sampleDT, "to_pydatetime") else row.sampleDT
    elevation = float(alt_m[i]) if alt_m is not None else None
    pt = gpxpy.gpx.GPXTrackPoint(float(row.lat), float(row.lon), elevation=elevation, time=ts)
    segment.points.append(pt)

# Write GPX
with open(output_file_name, "w") as f:
    f.write(gpx.to_xml())

print(f"Wrote {len(segment.points)} points to {output_file_name}")


Wrote 15881 points to output.gpx
